# ODL Assignment (Solution Development)
**Group number**: 15\
**Group leader**: Choong Man Shun (TP051283)\
**Group members**:
* Chew Cheng Yong (TP051338)
* Wong Poh Yee (TP051079)

# Table of Contents
[1. Exploratory Data Analysis (EDA)](#eda)\
[2. Data Preprocessing](#data_preprocessing)\
[3. Exploratory Data Analysis (EDA) (after data preprocessing)](#eda_after_data_preprocessing)\
[4. Model Implementation](#model_implementation)
- [4.1 ?](#model_1)
  - [4.1.1 Base Model](#base_model_1)
  - [4.1.2 Model Hyperparameter Tuning](#model_hyperparameter_tuning_1)
  - [4.1.3 Final Model](#final_model_1)
- [4.2 ?](#model_2)
  - [4.2.1 Base Model](#base_model_2)
  - [4.2.2 Model Hyperparameter Tuning](#model_hyperparameter_tuning_2)
  - [4.2.3 Final Model](#final_model_2)
- [4.3 ?](#model_3)
  - [4.3.1 Base Model](#base_model_3)
  - [4.3.2 Model Hyperparameter Tuning](#model_hyperparameter_tuning_3)
  - [4.3.3 Final Model](#final_model_3)

# 1. Exploratory Data Analysis (EDA) <a id="eda"/>

# 2. Data Preprocessing <a id="data_preprocessing"/>

# 3. Exploratory Data Analysis (EDA) (after data preprocessing) <a id="eda_after_data_preprocessing"/>

# 4. Model Implementation <a id="model_implementation"/>

## 4.1 ? <a id="model_1"/>

### 4.1.1 Base Model <a id="base_model_1"/>

### 4.1.2 Model Hyperparameter Tuning <a id="model_hyperparameter_tuning_1"/>

### 4.1.3 Final Model <a id="final_model_1"/>

## 4.2 ? <a id="model_2"/>

### 4.2.1 Base Model <a id="base_model_2"/>

### 4.2.2 Model Hyperparameter Tuning <a id="model_hyperparameter_tuning_2"/>

### 4.2.3 Final Model <a id="final_model_2"/>

## 4.3 ? <a id="model_3"/>

### 4.3.1 Base Model <a id="base_model_3"/>

### 4.3.2 Model Hyperparameter Tuning <a id="model_hyperparameter_tuning_3"/>

### 4.3.3 Final Model <a id="final_model_3"/>